In [ ]:
#設定Jupyter與Matplotlib交互方法

In [ ]:
%matplotlib inline

In [ ]:
#字串處理

In [ ]:
import pandas as pd

In [ ]:
def CSV_To_TXT(File_Path, Save_Path):
    Data = pd.read_csv(File_Path, header=None)
    Processed_Data = list(Data.iloc[:, 4])    #取出CSV檔案中的第五行並轉換成List
    with open(Save_Path, 'w') as Output: 
        Output.write((', '.join(map(str,Processed_Data))).replace(',', '\n').replace(' ', ''))   #將list去掉 "[]" 和空格，並將 "," 改成換行

In [ ]:
CSV_To_TXT("/home/chenwunsyuan/Documents/Python_EMG_Analysis_Project/Raw_Data/CSV/三頭肌持續出力.csv", "/home/chenwunsyuan/Documents/Python_EMG_Analysis_Project/Raw_Data/TXT/三頭肌持續出力.txt")
CSV_To_TXT("/home/chenwunsyuan/Documents/Python_EMG_Analysis_Project/Raw_Data/CSV/二頭肌持續出力.csv", "/home/chenwunsyuan/Documents/Python_EMG_Analysis_Project/Raw_Data/TXT/二頭肌持續出力.txt")
CSV_To_TXT("/home/chenwunsyuan/Documents/Python_EMG_Analysis_Project/Raw_Data/CSV/三頭肌最大出力.csv", "/home/chenwunsyuan/Documents/Python_EMG_Analysis_Project/Raw_Data/TXT/三頭肌最大出力.txt")
CSV_To_TXT("/home/chenwunsyuan/Documents/Python_EMG_Analysis_Project/Raw_Data/CSV/二頭肌最大出力.csv", "/home/chenwunsyuan/Documents/Python_EMG_Analysis_Project/Raw_Data/TXT/二頭肌最大出力.txt")
CSV_To_TXT("/home/chenwunsyuan/Documents/Python_EMG_Analysis_Project/Raw_Data/CSV/股二頭肌持續出力.csv", "/home/chenwunsyuan/Documents/Python_EMG_Analysis_Project/Raw_Data/TXT/股二頭肌持續出力.txt")
CSV_To_TXT("/home/chenwunsyuan/Documents/Python_EMG_Analysis_Project/Raw_Data/CSV/股二頭肌最大出力.csv", "/home/chenwunsyuan/Documents/Python_EMG_Analysis_Project/Raw_Data/TXT/股二頭肌最大出力.txt")
CSV_To_TXT("/home/chenwunsyuan/Documents/Python_EMG_Analysis_Project/Raw_Data/CSV/股內側肌持續出力.csv", "/home/chenwunsyuan/Documents/Python_EMG_Analysis_Project/Raw_Data/TXT/股內側肌持續出力.txt")
CSV_To_TXT("/home/chenwunsyuan/Documents/Python_EMG_Analysis_Project/Raw_Data/CSV/股內側肌最大出力.csv", "/home/chenwunsyuan/Documents/Python_EMG_Analysis_Project/Raw_Data/TXT/股內側肌最大出力.txt")
CSV_To_TXT("/home/chenwunsyuan/Documents/Python_EMG_Analysis_Project/Raw_Data/CSV/股外側肌持續出力.csv", "/home/chenwunsyuan/Documents/Python_EMG_Analysis_Project/Raw_Data/TXT/股外側肌持續出力.txt")
CSV_To_TXT("/home/chenwunsyuan/Documents/Python_EMG_Analysis_Project/Raw_Data/CSV/股外側肌最大出力.csv", "/home/chenwunsyuan/Documents/Python_EMG_Analysis_Project/Raw_Data/TXT/股外側肌最大出力.txt")

In [ ]:
#數據分析

In [ ]:
import math
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import biosppy

In [ ]:
#每次選取特定區間來計算RMS值

In [ ]:
def RMS_Plot(Input_Time, Input_Data, RMS_Sampling_Size):    #RMS_Plot(NumPy一維陣列, NumPy一維陣列, 數字)
    RMS_Array = np.zeros(0)    #建立空的一為陣列
    i = 0
    while i <= len(Input_Data):    #
        Temp1 = 0    #設定初始值
        Average_RMS = 0    #設定初始值
        for j in range(RMS_Sampling_Size):
            try:
                Temp1 += Input_Data[j + i] ** 2    #計算平方和
            except:
                j = j - 1    #
                break
        Average_RMS = math.sqrt(Temp1 / RMS_Sampling_Size) 
        RMS_Array = np.hstack((RMS_Array, np.full(j + 1, Average_RMS)))
        i += RMS_Sampling_Size
    
    plt.figure(figsize=(15,10))
    plt.plot(Input_Time, Input_Data, label='Filtered')
    plt.plot(Input_Time, RMS_Array, label='RMS')
    plt.legend()
    plt.title("EMG Summary With RMS")
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.show()

In [ ]:
def IEMG_Plot(Input_Time, Input_Data):
    IEMG_Array = np.zeros(0)
    for n in range(len(Input_Data)):
        try:
            Integrate_Result = sp.integrate.simps(np.abs(Input_Data[:n + 1]), Input_Time[:n + 1])
            IEMG_Array = np.hstack((IEMG_Array, np.full(1, Integrate_Result)))
        except:
            continue
    
    plt.figure(figsize=(15,10))
    plt.plot(Input_Time, Input_Data, label='Filtered')
    plt.plot(Input_Time, IEMG_Array, label='IEMG')
    plt.legend()
    plt.title("EMG Summary With IEMG")
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.show()
    #print(IEMS_Array)   

In [ ]:
def Plot_MPF(Input_Data, Sampling_Frequency):
    f, Pxx_den = sp.signal.welch(Input_Data, Sampling_Frequency)
    #f, Pxx_den = sp.signal.welch(Input_Data, Sampling_Frequency, nperseg=1024)
    
    plt.figure(figsize=(15,10))
    plt.semilogy(f, Pxx_den, label = 'MPF')
    plt.legend()
    plt.ylim([0.5e-3, 1])
    plt.xlabel('frequency [Hz]')
    plt.ylabel('MPF [V**2/Hz]')
    plt.show()

In [ ]:
#三頭肌持續出力分析

In [ ]:
Signal1 = np.loadtxt('/home/chenwunsyuan/Documents/Python_EMG_Analysis_Project/Raw_Data/TXT/三頭肌持續出力.txt')
Processed_Signal1 = biosppy.signals.emg.emg(signal=Signal1, sampling_rate=250, show=True)

In [ ]:
RMS_Plot(Processed_Signal1[0], Processed_Signal1[1], 15)

In [ ]:
IEMG_Plot(Processed_Signal1[0], Processed_Signal1[1])

In [ ]:
Plot_MPF(Processed_Signal1[1], 250)

In [ ]:
#三頭肌最大出力分析

In [ ]:
Signal2 = np.loadtxt('/home/chenwunsyuan/Documents/Python_EMG_Analysis_Project/Raw_Data/TXT/三頭肌最大出力.txt')
Processed_Signal2 = biosppy.signals.emg.emg(signal=Signal2, sampling_rate=250, show=True)

In [ ]:
RMS_Plot(Processed_Signal2[0], Processed_Signal2[1], 15)

In [ ]:
IEMG_Plot(Processed_Signal2[0], Processed_Signal2[1])

In [ ]:
Plot_MPF(Processed_Signal2[1], 250)

In [ ]:
#二頭肌持續出力分析

In [ ]:
Signal3 = np.loadtxt('/home/chenwunsyuan/Documents/Python_EMG_Analysis_Project/Raw_Data/TXT/二頭肌持續出力.txt')
Processed_Signal3 = biosppy.signals.emg.emg(signal=Signal3, sampling_rate=250, show=True)

In [ ]:
RMS_Plot(Processed_Signal3[0], Processed_Signal3[1], 15)

In [ ]:
IEMG_Plot(Processed_Signal3[0], Processed_Signal3[1])

In [ ]:
Plot_MPF(Processed_Signal3[1], 250)

In [ ]:
#二頭肌最大出力分析

In [ ]:
Signal4 = np.loadtxt('/home/chenwunsyuan/Documents/Python_EMG_Analysis_Project/Raw_Data/TXT/二頭肌最大出力.txt')
Processed_Signal4 = biosppy.signals.emg.emg(signal=Signal4, sampling_rate=250, show=True)

In [ ]:
RMS_Plot(Processed_Signal4[0], Processed_Signal4[1], 15)

In [ ]:
IEMG_Plot(Processed_Signal4[0], Processed_Signal4[1])

In [ ]:
Plot_MPF(Processed_Signal4[1], 250)

In [ ]:
#股二頭肌持續出力分析

In [ ]:
Signal5 = np.loadtxt('/home/chenwunsyuan/Documents/Python_EMG_Analysis_Project/Raw_Data/TXT/股二頭肌持續出力.txt')
Processed_Signal5 = biosppy.signals.emg.emg(signal=Signal5, sampling_rate=250, show=True)

In [ ]:
RMS_Plot(Processed_Signal5[0], Processed_Signal5[1], 15)

In [ ]:
IEMG_Plot(Processed_Signal5[0], Processed_Signal5[1])

In [ ]:
Plot_MPF(Processed_Signal5[1], 250)

In [ ]:
#股二頭肌最大出力分析

In [ ]:
Signal6 = np.loadtxt('/home/chenwunsyuan/Documents/Python_EMG_Analysis_Project/Raw_Data/TXT/股二頭肌最大出力.txt')
Processed_Signal6 = biosppy.signals.emg.emg(signal=Signal6, sampling_rate=250, show=True)

In [ ]:
RMS_Plot(Processed_Signal6[0], Processed_Signal6[1], 15)

In [ ]:
IEMG_Plot(Processed_Signal6[0], Processed_Signal6[1])

In [ ]:
Plot_MPF(Processed_Signal6[1], 250)

In [ ]:
#股內側肌持續出力分析

In [ ]:
Signal7 = np.loadtxt('/home/chenwunsyuan/Documents/Python_EMG_Analysis_Project/Raw_Data/TXT/股內側肌持續出力.txt')
Processed_Signal7 = biosppy.signals.emg.emg(signal=Signal7, sampling_rate=250, show=True)

In [ ]:
RMS_Plot(Processed_Signal7[0], Processed_Signal7[1], 15)

In [ ]:
IEMG_Plot(Processed_Signal7[0], Processed_Signal7[1])

In [ ]:
Plot_MPF(Processed_Signal7[1], 250)

In [ ]:
#股內側肌最大出力分析

In [ ]:
Signal8 = np.loadtxt('/home/chenwunsyuan/Documents/Python_EMG_Analysis_Project/Raw_Data/TXT/股內側肌最大出力.txt')
Processed_Signal8 = biosppy.signals.emg.emg(signal=Signal8, sampling_rate=250, show=True)

In [ ]:
RMS_Plot(Processed_Signal8[0], Processed_Signal8[1], 15)

In [ ]:
IEMG_Plot(Processed_Signal8[0], Processed_Signal8[1])

In [ ]:
Plot_MPF(Processed_Signal8[1], 250)

In [ ]:
#股外側肌持續出力分析

In [ ]:
Signal9 = np.loadtxt('/home/chenwunsyuan/Documents/Python_EMG_Analysis_Project/Raw_Data/TXT/股外側肌持續出力.txt')
Processed_Signal9 = biosppy.signals.emg.emg(signal=Signal9, sampling_rate=250, show=True)

In [ ]:
RMS_Plot(Processed_Signal9[0], Processed_Signal9[1], 15)

In [ ]:
IEMG_Plot(Processed_Signal9[0], Processed_Signal9[1])

In [ ]:
Plot_MPF(Processed_Signal9[1], 250)

In [ ]:
#股外側肌最大出力分析

In [ ]:
Signal10 = np.loadtxt('/home/chenwunsyuan/Documents/Python_EMG_Analysis_Project/Raw_Data/TXT/股外側肌最大出力.txt')
Processed_Signal10 = biosppy.signals.emg.emg(signal=Signal10, sampling_rate=250, show=True)

In [ ]:
RMS_Plot(Processed_Signal10[0], Processed_Signal10[1], 15)

In [ ]:
IEMG_Plot(Processed_Signal10[0], Processed_Signal10[1])

In [ ]:
Plot_MPF(Processed_Signal10[1], 250)